In [1]:
import pandas as pd
from T_data_loader import *

In [2]:
#Names of columns for the data IMDb
NAMES_CREW = ['IMDb_title_ID','IMDb_director_ID', 'IMDb_writers_ID']
NAMES_ratings = ['IMDb_title_ID', 'Average rating', 'number of votes'] 
NAMES_BASICS = ['IMDb_director_ID', 'Name']
NAMES_BASICS_2 = ['IMDb_title_ID', 'TitleType', 'Primary_title', 'Original_title', 'isAdult', 'realease_date', 'end_year', 'runtime', 'genres']
NAMES_NEW = ['IMDb_title_ID', 'Release_date','Runtime']

In [3]:
df_IMDb_crew = csv_loader('title.crew_Movie', 'IMDb', column_names=NAMES_CREW)
df_IMDb_ratings = csv_loader('title.ratings', 'IMDb', column_names=NAMES_ratings, header = 0)
df_IMDb_name = csv_loader('name.basics_Movie','IMDb', column_names=NAMES_BASICS)
df_IMDb_title = csv_loader('title.basics','IMDb', column_names=NAMES_BASICS_2, header = 0)
selected_type = 'movie'
df_IMDb_title = df_IMDb_title.query('TitleType==@selected_type')

In [4]:
print(df_IMDb_crew.shape)
print(df_IMDb_ratings.shape)
print(df_IMDb_name.shape)
print(df_IMDb_title.shape)

(679735, 3)
(1493420, 3)
(206512, 2)
(696692, 9)


## Get a Df with all known directors

In [5]:
df_IMDb_crew_director = df_IMDb_crew['IMDb_director_ID'].str.split(',', expand=True)

In [6]:
df_IMDb_crew_director.columns = [f'Director{i+1}' for i in range(df_IMDb_crew_director.shape[1])]

In [7]:
for i in range(df_IMDb_crew_director.shape[1]):
    director_col = f'Director{i+1}'
    
    df_IMDb_crew_director = df_IMDb_crew_director.merge(df_IMDb_name,left_on=director_col, right_on='IMDb_director_ID',how='left')

    df_IMDb_crew_director.rename(columns={'Name': f'{director_col}_name'}, inplace=True)
    df_IMDb_crew_director.drop(columns=[director_col], inplace=True)
    df_IMDb_crew_director.drop(columns=['IMDb_director_ID'], inplace=True)  

In [8]:
df_IMDb_crew_director=df_IMDb_crew_director.fillna('').apply(lambda row: ','.join(row.astype(str)), axis=1)

In [9]:
df_IMDb_crew_director=df_IMDb_crew_director.str.replace(',,', '', regex=False).str.lstrip(',').str.rstrip(',')

In [10]:
df_IMDb_crew_director = df_IMDb_crew_director.str.split(',', expand=True)

In [11]:
df_IMDb_crew_director=df_IMDb_crew_director.add_prefix("Director")

## Get a Df with all known writers

In [12]:
df_IMDb_crew_writers = df_IMDb_crew['IMDb_writers_ID'].str.split(',', expand=True)

In [13]:
df_IMDb_crew_writers.columns = [f'Writer{i+1}' for i in range(df_IMDb_crew_writers.shape[1])]

In [14]:
for i in range(df_IMDb_crew_writers.shape[1]):
    writer_col = f'Writer{i+1}'
    
    df_IMDb_crew_writers = df_IMDb_crew_writers.merge(df_IMDb_name,left_on=writer_col, right_on='IMDb_director_ID',how='left')

    df_IMDb_crew_writers.rename(columns={'Name': f'{writer_col}_name'}, inplace=True)
    df_IMDb_crew_writers.drop(columns=[writer_col], inplace=True)
    df_IMDb_crew_writers.drop(columns=['IMDb_director_ID'], inplace=True)  

In [15]:
df_IMDb_crew_writers=df_IMDb_crew_writers.fillna('').apply(lambda row: ','.join(row.astype(str)), axis=1)

In [16]:
df_IMDb_crew_writers=df_IMDb_crew_writers.str.replace(',,', '', regex=False).str.lstrip(',').str.rstrip(',')

In [17]:
df_IMDb_crew_writers = df_IMDb_crew_writers.str.split(',', expand=True)

In [18]:
df_IMDb_crew_writers=df_IMDb_crew_writers.add_prefix("Writer")

## Merging director, writer, rating 

In [19]:
complete_director_writer = pd.concat([df_IMDb_crew['IMDb_title_ID'],df_IMDb_crew_director,df_IMDb_crew_writers], axis = 1)

In [20]:
print(complete_director_writer.shape)

(679735, 98)


In [21]:
print(df_IMDb_ratings.shape)

(1493420, 3)


In [22]:
total_IMDb = pd.merge(complete_director_writer, df_IMDb_ratings, how='left', on = 'IMDb_title_ID' )

In [31]:
df_IMDb_title_select=df_IMDb_title[['IMDb_title_ID', 'realease_date','runtime','Primary_title', 'Original_title']]

In [32]:
complete_IMDb = pd.merge(total_IMDb, df_IMDb_title_select, how='outer', on = 'IMDb_title_ID' )

In [33]:
complete_IMDb.shape

(696856, 104)

In [34]:
complete_IMDb.to_pickle('IMDb/IMDb.pkl')